In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import snscrape.modules.twitter as sntwitter

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

from itertools import product

In [2]:
#nltk.download('wordnet')
#nltk.download('stopwords')

In [3]:
# Continuing adding the "tunable" parameters into this cell

KEY_PHRASES = ["climate change", "sustainability", "technology"]
START_DATE = "2021-01-01"
END_DATE = "2021-08-30"
MAX_TWEETS = 100

MIN_FOLLOWERS = 5000
MAX_TWEETS_PER_USER = 50

NUMBER_OF_TOPICS = 3
NO_TOP_WORDS = 3

In [4]:
key_words = [item for words in KEY_PHRASES for item in words.split(' ')]
key_word_sets = [item for word in key_words for item in wordnet.synsets(word)]

In [5]:
# Creating list to append tweet data to
tweet_list = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for key_phrase in KEY_PHRASES:
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(
                        key_phrase + ' since:' + START_DATE + ' until:' + END_DATE).get_items()):
        if i > MAX_TWEETS:
            break
        tweet_list.append([tweet.user.username, tweet.user.followersCount, tweet.user.verified])
# Creating a dataframe from the tweets list above
tweet_df = pd.DataFrame(tweet_list, columns=['Username', 'FollowerCount', 'Verified'])
tweet_df = tweet_df.loc[tweet_df['FollowerCount'] >= MIN_FOLLOWERS]
tweet_df = tweet_df.drop_duplicates(subset=['Username'])

In [6]:
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

def remove_links(tweet):
    '''Takes a string and removes web links from it'''
    tweet = re.sub(r'http\S+', '', tweet) # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet) # rempve bitly links
    tweet = tweet.strip('[link]') # remove [links]
    return tweet

def remove_users(tweet):
    '''Takes a string and removes retweet and @user information'''
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove retweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet) # remove tweeted at
    return tweet

# cleaning master function
def clean_tweet(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = tweet.lower() # lower case
    tweet = re.sub('['+my_punctuation + ']+', ' ', tweet) # strip punctuation
    tweet = re.sub('\s+', ' ', tweet) #remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    tweet_token_list = [word for word in tweet.split(' ')
                            if word not in my_stopwords] # remove stopwords

    tweet_token_list = [word_rooter(word) if '#' not in word else word
                        for word in tweet_token_list] # apply word rooter
    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]
    tweet = ' '.join(tweet_token_list)
    return tweet

In [7]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

In [10]:
candidate_usernames = tweet_df['Username'].values
user_max_scores = []
user_avg_scores = []
user_LDA_model = []

for twitter_user in candidate_usernames:
    
    #print(twitter_user)
    # Creating list to append tweet data to
    user_tweets = []

    # Using TwitterSearchScraper to scrape data and append tweets to list
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper('from:' + twitter_user).get_items()):
        if i > MAX_TWEETS_PER_USER:
            break
        user_tweets.append([tweet.date, tweet.id, tweet.content, tweet.user.username])

    # Creating a dataframe from the tweets list above 
    user_tweets_df = pd.DataFrame(user_tweets, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

    documents = list(user_tweets_df['Text'].values)
    cleaned_document = [clean_tweet(tweet) for tweet in documents]
    cleaned_document = [cleaned_tweet for cleaned_tweet in cleaned_document if cleaned_tweet != ' ']
    
    # the vectorizer object will be used to transform text to vector form
    vectorizer = CountVectorizer(token_pattern='\w+|\$[\d\.]+|\S+')

    # apply transformation
    tf = vectorizer.fit_transform(cleaned_document).toarray()

    # tf_feature_names tells us what word each column in the matric represents
    tf_feature_names = vectorizer.get_feature_names()
    
    model = LatentDirichletAllocation(n_components=NUMBER_OF_TOPICS, random_state=0)
    model.fit(tf)
    
    topic_df = display_topics(model, tf_feature_names, NO_TOP_WORDS)
    
    user_words = []
    for topic_number in range(NUMBER_OF_TOPICS):
        topic_words = topic_df["Topic " + str(topic_number) + " words"].values
        user_words += [word for word in topic_words]
        
    user_word_sets = [item for word in user_words for item in wordnet.synsets(word)]
    
    max_val = 0
    avg_val = 0
    count = 0
    #max_sets = ()
    for set_1, set_2 in product(key_word_sets, user_word_sets):
        similarity = set_1.wup_similarity(set_2)
        avg_val += similarity
        count += 1
        if similarity > max_val:
            max_val = similarity
            #max_sets = (set_1, set_2)
    if count > 0:
        avg_val /= count
    
    user_max_scores.append(max_val)
    user_avg_scores.append(avg_val)
    user_LDA_model.append(model)

TheNic02570495
WorldWarZeroOrg
Fox26Mike
EliLikesRice
mikesbloggity
AllanMargolin
AndyFillmoreHFX
KrisnBrit
RepMarieNewman
DoctorKarl
btsbaragi_jk
lizduckchong
SourceBottle
delphix
WesleyBKress
drkiki
JoeriRogelj
semodu_pr
unspinthespina1
RegisMedia
TheDXEngineer
Eavor
Greentekio
artarchi
Earth911
TatlerAsia
earthguardianz
RobTiffany
realmethod41
ScottTerrioHMA
RichardLehman1
Milkabot
va_cc11
MdRokivai4
tribunephl
Alevskey
jonathanstea
ventanaresearch
techweev
CioAmaro
SuriyaSubraman


In [11]:
tweet_df['Max Scores'] = user_max_scores
tweet_df['Avg Scores'] = user_avg_scores
tweet_df['LDA Model'] = user_LDA_model

In [12]:
tweet_df

,Username,FollowerCount,Verified,Max Scores,Avg Scores,LDA Model
0,TheNic02570495,6193,False,0.833333,0.234487,"LatentDirichletAllocation(n_components=3, rand..."
21,WorldWarZeroOrg,9539,True,0.923077,0.255239,"LatentDirichletAllocation(n_components=3, rand..."
29,Fox26Mike,24968,True,0.769231,0.233523,"LatentDirichletAllocation(n_components=3, rand..."
32,EliLikesRice,35047,False,0.941176,0.256972,"LatentDirichletAllocation(n_components=3, rand..."
34,mikesbloggity,44214,True,0.705882,0.250454,"LatentDirichletAllocation(n_components=3, rand..."
55,AllanMargolin,33039,False,0.666667,0.227893,"LatentDirichletAllocation(n_components=3, rand..."
61,AndyFillmoreHFX,15417,True,0.800000,0.259092,"LatentDirichletAllocation(n_components=3, rand..."
82,KrisnBrit,32596,False,0.933333,0.233519,"LatentDirichletAllocation(n_components=3, rand..."
91,RepMarieNewman,50115,True,0.800000,0.241570,"LatentDirichletAllocation(n_components=3, rand..."
95,DoctorKarl,337205,True,0.800000,0.252611,"LatentDirichletAllocation(n_components=3, rand..."


In [64]:
model.__dict__.keys()

dict_keys(['n_components', 'doc_topic_prior', 'topic_word_prior', 'learning_method', 'learning_decay', 'learning_offset', 'max_iter', 'batch_size', 'evaluate_every', 'total_samples', 'perp_tol', 'mean_change_tol', 'max_doc_update_iter', 'n_jobs', 'verbose', 'random_state', 'n_features_in_', 'random_state_', 'n_batch_iter_', 'n_iter_', 'doc_topic_prior_', 'topic_word_prior_', 'components_', 'exp_dirichlet_component_', 'bound_'])

In [27]:
tweet_list[0].user.__dict__.keys()

dict_keys(['username', 'id', 'displayname', 'description', 'rawDescription', 'descriptionUrls', 'verified', 'created', 'followersCount', 'friendsCount', 'statusesCount', 'favouritesCount', 'listedCount', 'mediaCount', 'location', 'protected', 'linkUrl', 'linkTcourl', 'profileImageUrl', 'profileBannerUrl', 'label'])